In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ======================= 1. Install & Import ======================
# import torch
# print("Torch version:", torch.__version__)
# print("CUDA available:", torch.cuda.is_available())
# print("CUDA device count:", torch.cuda.device_count())


In [3]:
import os
import timm
import torch
import torch.nn as nn
from torchvision import transforms
# from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import torch.multiprocessing as mp
# from dataset import Stanford40Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

# Class
from PIL import Image
from torch.utils.data import Dataset

# for report
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

In [4]:
# ======================= 2. Custom Dataset ========================
class Stanford40Dataset(Dataset):
    def __init__(self, img_dir, split_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(split_file) as f:
            # danh sách tên ảnh (không .jpg)
            self.names = [l.strip().split(".")[0] for l in f if l.strip().split(".")[0]]
        # build list nhãn đầy đủ
        labels = sorted({self._label_from_name(n) for n in self.names})
        self.cls2idx = {c:i for i,c in enumerate(labels)}

    def _label_from_name(self, name):
        parts = name.split("_")
        return "_".join(parts[:-1])  # hoặc " ".join(parts[:-1])

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img_path = os.path.join(self.img_dir, name + ".jpg")
        img = Image.open(img_path).convert("RGB")
        label_str = self._label_from_name(name)
        label = self.cls2idx[label_str]
        if self.transform:
            img = self.transform(img)
        return img, label

In [5]:
# ======================= 3. Hyperparams & Transforms ==============
NUM_CLASSES = 40
BATCH_SIZE = 16
LR = 1e-4 # Learning Rate
WD = 2e-2 # Weigth decay
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(
        brightness=0.4, 
        contrast=0.4, 
        saturation=0.4, 
        hue=0.1
    ),
    transforms.RandomRotation(degrees=10),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406), 
        std=(0.229, 0.224, 0.225)
    ),
    transforms.RandomErasing(
        p=0.5, 
        scale=(0.02, 0.33), 
        ratio=(0.3, 3.3), 
        value='random'
    ),
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])

In [6]:
# ======================= 4. DataLoader ============================
DATA_ROOT = "/kaggle/input/stanford40"
train_ds = Stanford40Dataset(
    img_dir=os.path.join(DATA_ROOT, "JPEGImages"),
    split_file=os.path.join(DATA_ROOT, "ImageSplits", "train.txt"),
    transform=train_transform)
val_ds = Stanford40Dataset(
    img_dir=os.path.join(DATA_ROOT, "JPEGImages"),
    split_file=os.path.join(DATA_ROOT, "ImageSplits", "test.txt"),
    transform=val_transform)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(train_loader.__len__)
print(val_loader.__len__)

<bound method DataLoader.__len__ of <torch.utils.data.dataloader.DataLoader object at 0x783f38a44790>>
<bound method DataLoader.__len__ of <torch.utils.data.dataloader.DataLoader object at 0x783f38a6fa50>>


In [7]:
# =============== 5. Model ===============
# model = timm.create_model("convit_base", pretrained=True)

# # Freeze toàn bộ parameters trước
# for param in model.parameters():
#     param.requires_grad = False

# # Thay classifier head
# in_features = model.head.in_features
# model.head = nn.Linear(in_features, NUM_CLASSES)

# # Head mới có requires_grad=True theo mặc định
# model = model.to(DEVICE)
model = timm.create_model("convit_base", pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Unfreeze 2 block Transformer (10, 11)
for idx in [10, 11]:
    for param in model.blocks[idx].parameters():
        param.requires_grad = True

in_features = model.head.in_features
model.head = nn.Linear(in_features, NUM_CLASSES)
model = model.to(DEVICE)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [8]:
# ======================= 6. Optimizer, Scheduler, Loss ===========
optimizer = torch.optim.SGD(model.parameters(), momentum = 0.9,
                               lr=LR, weight_decay= WD)
# cosine + 5-epoch warm-up
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
              optimizer, T_max=30,
              eta_min=1e-6)

criterion = nn.CrossEntropyLoss()


In [ ]:
# ======================= 7. Training Loop ========================
train_losses = []
val_losses   = []
val_accs     = []

NUM_EPOCHS = 30 # try 30 epoch when freeze head

best_acc = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    # ======== Training ========
    model.train()
    running_train_loss = 0.0
    num_train_samples = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = imgs.size(0)
        running_train_loss += loss.item() * batch_size
        num_train_samples += batch_size

    avg_train_loss = running_train_loss / num_train_samples
    train_losses.append(avg_train_loss)
    scheduler.step()

    # ======== Validation ========
    model.eval()
    running_val_loss = 0.0
    num_val_samples = 0
    correct = 0

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [ Val ]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            batch_size = imgs.size(0)
            running_val_loss += loss.item() * batch_size
            num_val_samples += batch_size

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    avg_val_loss = running_val_loss / num_val_samples
    val_losses.append(avg_val_loss)

    acc = correct / num_val_samples
    val_accs.append(acc)

    print(
        f"Epoch {epoch}/{NUM_EPOCHS} | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Loss: {avg_val_loss:.4f} | "
        f"Val Acc: {acc*100:.2f}%"
    )

    # ======== Save best ========
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_convit_stanford40.pth")

print(f"Best Val Acc only head: {best_acc*100:.2f}%")

Epoch 1/30 [ Val ]: 100%|██████████| 346/346 [00:45<00:00,  7.67it/s]


Epoch 1/30 | Train Loss: 3.5783 | Val Loss: 3.3367 | Val Acc: 19.58%


Epoch 2/30 [ Val ]: 100%|██████████| 346/346 [00:44<00:00,  7.69it/s]


Epoch 2/30 | Train Loss: 3.1588 | Val Loss: 2.8760 | Val Acc: 45.66%


Epoch 3/30 [ Val ]: 100%|██████████| 346/346 [00:44<00:00,  7.70it/s]


...

Epoch 8/30 [ Val ]: 100%|██████████| 346/346 [00:44<00:00,  7.70it/s]


Epoch 8/30 | Train Loss: 1.4471 | Val Loss: 1.2123 | Val Acc: 77.64%


Epoch 9/30 [ Val ]: 100%|██████████| 346/346 [00:44<00:00,  7.71it/s]


Epoch 9/30 | Train Loss: 1.3321 | Val Loss: 1.1137 | Val Acc: 78.56%


Epoch 10/30 [ Val ]:   2%|▏         | 6/346 [00:00<00:46,  7.26it/s]

In [ ]:
# ======================= 8. Loss and Accuracy ========================
epochs = np.arange(1, NUM_EPOCHS + 1)

plt.figure(figsize=(12,5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses,   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, np.array(val_accs)*100, label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Validation Accuracy over Epochs")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# ======== 9. Confusion Matrix and Classification report ========
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in tqdm(val_loader, desc="Final Eval [Val]"):
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# ======================= 6. Confusion Matrix and heatmap ========================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in tqdm(val_loader, desc="Final Eval [Val]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

cm = confusion_matrix(all_labels, all_preds)

# Draw heatmap confusion matrix with matplotlib
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion Matrix (Stage 2)")
plt.colorbar()
tick_marks = np.arange(NUM_CLASSES)

plt.xticks(tick_marks, classes, rotation=90, fontsize=7)
plt.yticks(tick_marks, classes, fontsize=7)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# 2) Classification report
# Cần list tên các class (40 labels) theo đúng thứ tự cls2idx
classes = [c for c, idx in sorted(train_ds.cls2idx.items(), key=lambda x: x[1])]
report = classification_report(all_labels, all_preds, target_names=classes, digits=4)
print("=== Classification Report ===")
print(report)

**1. Quan sát chung**
**Accuracy chung:** 84.83 % trên toàn bộ 5 532 ảnh validation.

* **Macro F1-score:** khoảng 0.8385, nghĩa là nếu lấy trung bình F1 của 40 lớp (không tính trọng số), mô hình đạt ~83.85 %.

* **Weighted F1-score:** ~0.8495, có phần cao hơn macro, vì các lớp nhiều support (ví dụ “riding_a_bike” 193 ảnh, “walking_the_dog” 193 ảnh,…) có F1 tốt hơn, kéo điểm trung bình lên.

Nhìn tổng thể, mô hình hoạt động khá tốt với nhiều lớp F1 ≥ 0.90, nhưng vẫn còn một số lớp F1 rất thấp (dưới 0.60), cần xem xét để cải thiện.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# === 9.1. TOP-5 maximum value in confusion matrix ===

# cm : numpy array (40, 40)
# create list tuple (count, true_idx, pred_idx)
flat_cm = [
    (int(cm[i, j]), i, j)
    for i in range(cm.shape[0])
    for j in range(cm.shape[1]) if (i != j)
]

# sort decs
flat_cm.sort(key=lambda x: x[0], reverse=True)

print("=== Top 20 values in Confusion Matrix ===")
for count, i, j in flat_cm[:20]:
    print(f"Count={count:4d} | True = '{classes[i]:<20}' | Pred = '{classes[j]:<20}'")
# Eg: Count= 123 | True = 'walking            ' | Pred = 'walking            '

**2. Phân tích Top-20 giá trị lớn nhất trong Confusion Matrix**
Những cặp (true→pred) có count cao cho thấy mô hình thường nhầm lẫn giữa hai lớp đó:

* **(applauding → waving_hands) = 25 lần**

    - “applauding” (vỗ tay) và “waving_hands” (vẫy tay) đều liên quan đến hai bàn tay chuyển động, rất dễ bị nhầm.

    - Khi nhìn một người co bàn tay lên, mô hình khó phân biệt là họ đang vỗ tay hay đang vẫy chào.

* **(phoning → smoking) = 24 lần**

    - “phoning” (đang cầm điện thoại) và “smoking” (đang cầm điếu thuốc) đều có động tác đưa bàn tay lên miệng/gần miệng, bối cảnh tương tự (tay cầm vật nhỏ, ngước mắt nhìn).

    - Nếu background không rõ (ví dụ tay che mặt, màu da, ánh sáng…), model dễ nhầm.

* **(reading → writing_on_a_book) = 23 lần**

    - “reading” (đang đọc sách) và “writing_on_a_book” (đang viết vào sách) đôi khi rất giống: cả hai đều đưa đầu về phía sách, tay bám vào bìa/cuốn sách.

    - Khi ảnh không cho thấy rõ cây bút, model chỉ nhìn thấy “người gập đầu vào sách” nên không phân biệt được.

* **(cooking → washing_dishes) = 19 lần, (cooking → cutting_vegetables) = 16 lần**

    - “cooking” (nấu ăn) thường diễn ra trong bếp, xung quanh có dao thớt, nồi, cả bồn rửa bát…

    - “washing_dishes” (rửa bát) và “cutting_vegetables” (thái rau) đều là một phần của quy trình nấu ăn. Khi ảnh chỉ thấy người đứng cạnh bồn rửa, model dễ gán nhầm thành “washing_dishes” hoặc “cutting_vegetables” thay vì “cooking”.

* **(waving_hands → applauding) = 16 lần**

    - Vỗ tay và vẫy tay thường khó phân biệt chỉ qua một khung tĩnh.

* **Một số cặp nhầm lẫn khác đáng chú ý:**

    - (drinking → pouring_liquid) = 11

    - (reading → texting_message) = 11

    - (smoking → phoning) = 11, (phoning → taking_photos) = 10

    - (running → throwing_frisby) = 10: hai hoạt động này đều có cử động tay chân tương tự, đặc biệt là khi ảnh chụp lúc chạy kèm theo động tác “ném” (tay vung ra).

    - (jumping → running) = 9: đôi khi khó phân biệt tư thế nhảy với tư thế đang chạy chỉ qua một ảnh tĩnh (chân, dáng người).

* **Những cặp trên phản ánh hai vấn đề chính:**

    - Các hành động quá “lý thuyết giống nhau” về tư thế (pose) và bối cảnh (background).

    - Một số lớp thực sự nằm trong cùng tổng thể ngữ cảnh (vd “cooking” bao gồm cắt, rửa, xào, v.v.) dẫn đến nhầm lẫn.



In [ ]:
# === 9.2. F1-score for all class and print top10 highest / lowest ===

# precision, recall, f1_scores, support
precision, recall, f1_scores, support = precision_recall_fscore_support(
    all_labels, 
    all_preds,
    labels=range(len(classes)),
    zero_division=0  
)

# list tuple (f1, class_idx)
f1_with_idx = [(f1_scores[i], i) for i in range(len(classes))]

# Top-10 F1 highest
f1_with_idx.sort(key=lambda x: x[0], reverse=True)
print("\n=== Top 10 highest F1-scores ===")
for f1, idx in f1_with_idx[:10]:
    print(f"Class = '{classes[idx]:<20}' | F1 = {f1:.4f}")


In [ ]:
# Top-10 F1 lowest
f1_with_idx.sort(key=lambda x: x[0]) 
print("\n=== Top 10 lowest F1-scores ===")
for f1, idx in f1_with_idx[:10]:
    print(f"Class = '{classes[idx]:<20}' | F1 = {f1:.4f}")

In [ ]:
all_with_idx = [(precision[i], recall[i], f1_scores[i], support[i], i) for i in range(len(classes))]
all_with_idx.sort(key=lambda x: x[2]) 
for pre, rec, f1, sup, idx in all_with_idx[:10]:
    print(f"Class = '{classes[idx]:<20}' | precision = {pre:.4f} | recall = {rec:.4f} | F1 = {f1:.4f} | support = {sup}")

**4. Phân tích Top-10 F1-score thấp nhất**
Nhận xét:
**waving_hands (F1=0.5645)**

        * Precision chỉ 50.7 %: tức hơn 49 % ảnh mà model gán nhãn “waving_hands” hóa ra là các lớp khác (FP nhiều).

        * Recall 63.6 %: model cũng bỏ sót ~36 % ảnh thực là “waving_hands” (FN kha khá).

        * Support = 110: đủ dùng để đánh giá, không quá ít.

    Nguyên nhân chính:

        * “vẫy tay” và “vỗ tay” (applauding) rất dễ nhầm (như nhìn qua confusion matrix, 16 lần waving_hands → applauding, 25 lần applauding → waving_hands).

        * Các dáng tay, góc chụp, background đa dạng (có thể ngoài trời, trong nhà, gần mặt, xa mặt) – mô hình khó tìm đặc trưng ổn định.

**texting_message (F1=0.5688)**

        * Precision 49.6 %: tức nửa số ảnh model dự đoán “texting_message” đều không phải (FP).
        
        * Recall 66.7 %: model bỏ sót ~33 % ảnh thực (FN).
        
        * Support = 93: không quá ít.

    Nguyên nhân:

        * “texting_message” (gõ SMS) và “reading” hay “phoning” hay “using_a_computer” đều có cử chỉ tương tự: người cúi nhìn màn hình nhỏ cầm trên tay.
        
        * Thiếu cue rõ (màn hình điện thoại vs. mặt bàn phím) vì ảnh tĩnh thường làm mờ chi tiết.
        
        * Bối cảnh: có thể ngoài đường, trong nhà, rất khó phân biệt.

**phoning (F1=0.5772)**

    * Precision 61.9 %, Recall 54.1 %.
    
    * Rất nhiều nhầm lẫn với “smoking” (24 lần) và “taking_photos” (10 lần).
    
    * Khi người cầm điện thoại che gần miệng, giống như cầm điếu thuốc hoặc cầm máy ảnh.

**pouring_liquid (F1=0.6502), smoking (F1=0.6549), taking_photos (F1=0.6605)**

Tất cả đều có gestures “tay đưa lên/vào”.

* “pouring_liquid” (rót nước) và “washing_dishes” (rửa bát) đôi khi nhầm lẫn (đã chart confusion 8 lần).

* “smoking” và “phoning” liên tiếp nhầm cho nhau: mỗi bên ~11 lần.

* “taking_photos” (chụp ảnh) vs “phoning” vs “looking_through_telescope/ -microscope”: cả ba đều đưa tay cầm thiết bị sát mặt, khó phân biệt.

**applauding (F1=0.6627)**

    * Precision 72.7 % nhưng Recall chỉ 60.9 %.
    
    * Nhầm với “waving_hands” (25 lần) và ngược lại 16 lần.

**washing_dishes (F1=0.6769)**

    * Precision 58.4 % (FP lớn), Recall 80.5 %.
    
    * Nhầm với “pouring_liquid” (8 lần) hoặc “cooking” (19 lần).
    
    * Background bếp nhà, bàn bồn rửa, dao thớt, nồi chảo dễ trùng với cảnh “cooking”.

**reading (F1=0.7033), drinking (F1=0.7055)**

    * Precision ~75 %, Recall ~66 %.

    * “reading” nhầm với “writing_on_a_book” (23 lần) hoặc “texting_message” (11 lần).
    
    * “drinking” nhầm với “pouring_liquid” (11 lần), “phoning” (9 lần).
    
    * Hành động cúi sát xuống tô chén hoặc cầm ly lên gần miệng rất dễ nhầm.

**Kết luận:**

**1. Những hành động “tay đưa lên/vào” dễ nhầm lẫn lẫn nhau**

Các lớp như waving_hands, applauding, phoning, smoking, taking_photos, pouring_liquid đều chia sẻ tư thế tương tự—tay giơ lên gần mặt hoặc sang ngang.

Kết quả:

* waving_hands–applauding: nhầm nhau tới 25 và 16 lần.

* phoning–smoking: nhầm nhau 24 lần.

* phoning–taking_photos: nhầm 10 lần.

* pouring_liquid–washing_dishes: nhầm 8 lần.

Hậu quả: Precision và Recall của từng lớp đều giảm mạnh (F1 khoảng 0.56–0.66).

**2. Thiếu “cue” rõ rệt để phân biệt khi chỉ có ảnh tĩnh**

* Ví dụ, để phân biệt “texting_message” và “reading” hay “phoning”, model phải nhìn thấy rõ màn hình điện thoại hoặc bàn phím. Ảnh tĩnh thường không đủ chi tiết đó.

* Khi “drinking” và “pouring_liquid” cả hai có cử chỉ đưa ly lên, model dễ bỏ sót (Recall ~66 %) hoặc gán nhầm (Precision ~64 %).

**3. Background (“bối cảnh chung”) góp phần gây nhiễu**

* “washing_dishes” và “cooking” đều diễn ra trong bếp, với bồn rửa, dao thớt, nồi chảo. Model thường chỉ dùng bối cảnh (bếp) để phán đoán, dẫn đến nhầm lẫn 19 lần “cooking → washing_dishes” và 8 lần “washing_dishes → pouring_liquid”.

* “reading”–“writing_on_a_book”: cả hai đều xuất hiện trong khung sách, nên nhầm lẫn 23 lần.

**4. Một số lớp thực hiện động tác đặc trưng rõ ràng, do đó model đã học rất tốt**

* Các hoạt động có tư thế rất đặc trưng như “playing_violin”, “playing_guitar”, “shooting_an_arrow”, “riding_a_bike”, “riding_a_horse” (F1 ≥ 0.95) hầu như không bị nhầm với lớp khác.

**Hướng cải tiến:**

**1. Tăng cường thông tin pose/hand-object**

* Thêm nhánh pose (keypoint) hoặc heatmap tay để phân biệt cầm khẩu súng, cầm đàn hay cầm ly, cầm điện thoại.

* Dùng một model nhỏ phát hiện “board” (trong reading vs. writing), “phone” vs. “cigarette” vs. “camera” để làm feature phụ.

**2. Định vị vùng quan trọng (ROI)**

* Crop khu vực tay-vật (dựa trên bounding-box phát hiện object) để model chú trọng các chi tiết nhỏ.

* Ví dụ với “phoning”, bắt vùng tay–mặt; với “pouring_liquid”–“washing_dishes”, bắt vùng bồn và cốc chén.

**3. Data augmentation mạnh với các tình huống dễ nhầm**

* Tạo thêm ảnh “vỗ tay”/“vẫy tay” với nhiều background khác nhau, góc chụp nhiều hướng.

* Sinh thêm ảnh “cầm điện thoại” không phải phoning (ví dụ selfie) để giảm FP.

* Thêm ảnh “cầm điếu thuốc” đa dạng, “cầm máy ảnh” ở nhiều góc.

**4. Weighted loss hoặc oversampling**

* Tăng trọng số (loss weight) cho các lớp F1 thấp (ví dụ waving_hands, texting_message, phoning) để model giảm FP/FN.

* Hoặc oversample bộ ảnh của những lớp này, giúp model có nhiều ví dụ hơn để phân biệt.

**5. Fine-tune thêm:**

* Train head + vài block cuối, để thu hẹp khoảng cách domain từ ImageNet.

* Unfreeze toàn bộ backbone, train thêm với learning rate cực thấp, tránh overfit.

In [ ]:
from sklearn.metrics import average_precision_score

model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)                 # logits shape (B,40)
        probs = torch.softmax(outputs, 1).cpu().numpy()  # (B,40)
        all_probs.append(probs)
        all_labels.append(labels.numpy())

# Ghép lại thành (N,40) và (N,)
all_probs = np.concatenate(all_probs, axis=0)   # (N,40)
all_labels = np.concatenate(all_labels, axis=0) # (N,)

# Tạo mat nhãn one-hot (N,40)
N, C = all_probs.shape
y_true_mat = np.zeros((N, C), dtype=np.int32)
y_true_mat[np.arange(N), all_labels] = 1

# Tính AP cho từng lớp
APs = np.zeros(C, dtype=np.float32)
for c in range(C):
    APs[c] = average_precision_score(y_true_mat[:, c], all_probs[:, c])

mAP = APs.mean()
print("=> mAP = {:.2f}%".format(mAP * 100.0))

# Unfreeze more block on backbone  

In [ ]:
# ======================= 1. Load model + head-only checkpoint (stage 1) ========================
model = timm.create_model("convit_base", pretrained=False)
in_features = model.head.in_features
model.head = nn.Linear(in_features, NUM_CLASSES)
model = model.to(DEVICE)

CHECKPOINT_PATH = "/kaggle/working/best_convit_stanford40.pth"
checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint)
print("Loaded checkpoint.")


# ======================= 2. Unfreeze backbone (stage 2) ========================
for param in model.parameters():
    param.requires_grad = False

for idx in [8, 11]:
    for param in model.blocks[idx].parameters():
        param.requires_grad = True

for param in model.head.parameters():
    param.requires_grad = True

# ======================= 3. Optimizer, Scheduler, Criterion ========================
LR = 2e-4
WEIGHT_DECAY = 3e-2
TOTAL_EPOCHS = 50
STAGE1_EPOCHS = NUM_EPOCHS
STAGE2_EPOCHS = TOTAL_EPOCHS - STAGE1_EPOCHS  # = 20

optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum = 0.9, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=STAGE2_EPOCHS, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()


In [ ]:
# ======================= 4. Stage 2 Training Loop + Save metrics ========================
train_losses = []
val_losses   = []
val_accs     = []

best_acc = 0.0
patience = 10
epochs_no_improve = 0

for epoch2 in range(1, STAGE2_EPOCHS + 1):
    epoch = STAGE1_EPOCHS + epoch2
    model.train()
    running_train_loss = 0.0
    total_train = 0

    for imgs, labels in tqdm(train_loader, desc=f"[Stage2] Epoch {epoch}/{TOTAL_EPOCHS} [Train]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        bs = imgs.size(0)
        running_train_loss += loss.item() * bs
        total_train += bs

    avg_train_loss = running_train_loss / total_train
    train_losses.append(avg_train_loss)
    scheduler.step()

    # Validation
    model.eval()
    running_val_loss = 0.0
    total_val = 0
    correct = 0

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"[Stage2] Epoch {epoch}/{TOTAL_EPOCHS} [ Val ]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            bs = imgs.size(0)
            running_val_loss += loss.item() * bs
            total_val += bs

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    avg_val_loss = running_val_loss / total_val
    val_losses.append(avg_val_loss)

    val_acc = correct / total_val
    val_accs.append(val_acc)

    print(
        f"Epoch {epoch}/{TOTAL_EPOCHS} | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Loss: {avg_val_loss:.4f} | "
        f"Val Acc: {val_acc*100:.2f}% | "
        f"LR: {scheduler.get_last_lr()[0]:.6f}"
    )

    # Early stopping & lưu model tốt nhất
    if val_acc > best_acc:
        best_acc = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_convit_stage2.pth")
        print(f"  → Saved new best model at epoch {epoch} (Val Acc={val_acc*100:.2f}%)")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"  → No improvement for {patience} epochs → Early stopping at epoch {epoch}")
            break

print(f"Finished Stage 2. Best Val Acc: {best_acc*100:.2f}%")

In [ ]:

# ======================= 5. Loss & Accuracy ========================
epochs = np.arange(STAGE1_EPOCHS+1, STAGE1_EPOCHS + len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# Train & Val Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses,   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Stage 2: Loss over Epochs")
plt.legend()

# Val Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, np.array(val_accs) * 100, label="Val Acc (%)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Stage 2: Val Accuracy over Epochs")
plt.legend()

plt.tight_layout()
plt.show()

# ======================= 6. Confusion Matrix and heatmap ========================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in tqdm(val_loader, desc="Final Eval [Val]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

cm = confusion_matrix(all_labels, all_preds)

# Draw heatmap confusion matrix with matplotlib
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion Matrix (Stage 2)")
plt.colorbar()
tick_marks = np.arange(NUM_CLASSES)

plt.xticks(tick_marks, classes, rotation=90, fontsize=7)
plt.yticks(tick_marks, classes, fontsize=7)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

Khi unfreeze toàn bộ mô hình, hiện tượng overfitting xảy ra trên tập huấn luyện. Do đó, tôi sẽ thử chỉ unfreeze một vài lớp cuối của khối Vision Transformer, như đề xuất trong bài báo, nhằm giúp mô hình học được đặc trưng phân bố hình ảnh liên quan đến hành động.

In [ ]:
print(model)

In [ ]:
model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)                 # logits shape (B,40)
        probs = torch.softmax(outputs, 1).cpu().numpy()  # (B,40)
        all_probs.append(probs)
        all_labels.append(labels.numpy())

# Ghép lại thành (N,40) và (N,)
all_probs = np.concatenate(all_probs, axis=0)   # (N,40)
all_labels = np.concatenate(all_labels, axis=0) # (N,)

# Tạo mat nhãn one-hot (N,40)
N, C = all_probs.shape
y_true_mat = np.zeros((N, C), dtype=np.int32)
y_true_mat[np.arange(N), all_labels] = 1

# Tính AP cho từng lớp
APs = np.zeros(C, dtype=np.float32)
for c in range(C):
    APs[c] = average_precision_score(y_true_mat[:, c], all_probs[:, c])

mAP = APs.mean()
print("=> mAP = {:.2f}%".format(mAP * 100.0))